In [2]:
from __future__ import print_function
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

np.random.seed(0)
x,y = np.random.multivariate_normal([0,0],[[1,0],[0,1]],100).T
data1 = np.append(x.reshape(100,1),y.reshape(100,1),axis=1)
labels1 = (np.ones(100))*-1
x,y = np.random.multivariate_normal([0.5,0.5],[[1,-1],[-1,2]],100).T
data2 = np.append(x.reshape(100,1),y.reshape(100,1),axis=1)
labels2 = np.ones(100)
data = np.append(data1,data2, axis=0)
labels = np.append(labels1, labels2, axis=0)
labels = labels.reshape(200,1)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
plt.plot(data1[:,:1],data1[:,1:],'x')
plt.plot(data2[:,:1],data2[:,1:],'x')
plt.axis('equal'); plt.show()

<Figure size 640x480 with 1 Axes>

In [3]:
def sigmoid(x):
    return 1/(1+np.exp(-(x)))


np.random.seed(1)
weights1 = np.random.rand(2,2)
weights2 = np.random.rand(2,1)
eta = 0.01
print(weights1, weights2)

for i in range(50):
    hidden_states =  np.tanh(np.dot(X_train, weights1))
    output_state = sigmoid(np.dot(hidden_states, weights2))
    error_out = np.square(output_state - y_train)/2
    slope_out = sigmoid(output_state)*(1-sigmoid(output_state))
    slope_hid = 1 - (hidden_states)*(hidden_states)
    d_output = error_out * slope_out
    error_hid = np.dot(d_output, weights2.T)
    d_hid = error_hid * slope_hid
    weights2 = weights2 + np.dot(hidden_states.T, d_output)*eta
    weights1 = weights1 + np.dot(X_train.T, d_hid)*eta

print(weights1, weights2)

###Testing 
hidden_states =  np.tanh(np.dot(X_test, weights1))
output_state = sigmoid(np.dot(hidden_states, weights2))



[[4.17022005e-01 7.20324493e-01]
 [1.14374817e-04 3.02332573e-01]] [[0.14675589]
 [0.09233859]]
[[0.85907666 1.05326801]
 [0.81683811 1.04164087]] [[1.56722385]
 [1.97508092]]


In [6]:
import torch
import torch.nn as nn


#data
X = torch.tensor(X_train, dtype=torch.float) 
y = torch.tensor(y_train, dtype=torch.float) 
xPredicted = torch.tensor(X_test, dtype=torch.float)


In [54]:
class Neural_Network(nn.Module):
    def __init__(self, ):
        super(Neural_Network, self).__init__()
        # parameters
        # TODO: parameters can be parameterized instead of declaring them here
        self.inputSize = 2
        self.outputSize = 1
        self.hiddenSize = 2
        self.eta = 0.01
        
        # weights
        self.W1 = torch.randn(self.inputSize, self.hiddenSize) 
        self.W2 = torch.randn(self.hiddenSize, self.outputSize)
        self.B1 = torch.randn(self.hiddenSize)
        self.B2 = torch.randn(self.outputSize)
        
    def forward(self, X):
        self.z = torch.matmul(X, self.W1) + self.B1 # 3 X 3 ".dot" does not broadcast in PyTorch
        self.z2 = self.tanh(self.z) # activation function
        self.z3 = torch.matmul(self.z2, self.W2) + self.B2
        o = self.sigmoid(self.z3) # final activation function
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def tanh(self, s):
        return (2/(1+torch.exp(-2*s))) - 1
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)
    
    def tanhPrime(self, s):
        #derivative of tanh
        return 1 - (s*s)
    
    def backward(self, X, y, o):
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # derivative of sig to error
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.tanhPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta) * self.eta
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta) * self.eta
        self.B1 += torch.sum(self.z2_delta, axis=0) * self.eta
        self.B2 += torch.sum(self.o_delta, axis=0) * self.eta
        
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        params = model.parameters()
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        

In [71]:
from sklearn.metrics import accuracy_score

NN = Neural_Network()
for i in range(1000):  # trains the NN 1,000 times
    NN.train(X, y)

NN.saveWeights(NN)
output = NN.forward(xPredicted)
y_pred = []

for i in range(len(output)):
    if(output[i]>0.1):
        y_pred.append(1)
    else:
        y_pred.append(-1)
        
acc = accuracy_score(y_test, y_pred)
print(acc)

0.7


In [25]:
import matplotlib.pyplot as plt
from torch import nn, optim

In [26]:
input_size = 2
hidden_size = 2
output_size = 1

model = nn.Sequential(nn.Linear(input_size, hidden_size),
                      nn.Tanh(),
                      nn.Linear(hidden_size, output_size),
                      nn.Sigmoid())
print(model)

Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Tanh()
  (2): Linear(in_features=2, out_features=1, bias=True)
  (3): Sigmoid()
)


In [51]:
criterion = nn.MSELoss()
optimizer = optim.Adagrad(model.parameters(), lr=0.01)
epochs = 100
for e in range(epochs):
    running_loss = 0
    for i in range(X_train.shape[0]):
    
        # Training pass
        optimizer.zero_grad()
        
        output = model(X[i])
        loss = criterion(output, y[i])
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()
    if(e%10==0):    
        print("Epoch {} - Training loss: {}".format(e, running_loss/X_train.shape[0]))

Epoch 0 - Training loss: 0.9360702644102276
Epoch 10 - Training loss: 0.9350921055302024
Epoch 20 - Training loss: 0.9349891745485366
Epoch 30 - Training loss: 0.9349286315962673
Epoch 40 - Training loss: 0.9348854111507535
Epoch 50 - Training loss: 0.9348518570885063
Epoch 60 - Training loss: 0.9348245237022639
Epoch 70 - Training loss: 0.9348015260882676
Epoch 80 - Training loss: 0.9347816589288414
Epoch 90 - Training loss: 0.9347643310204148


In [53]:
y_pred = []
with torch.no_grad():
    for i in range(X_test.shape[0]):
        logps = model(xPredicted[i])
        if(logps>0.001):
            y_pred.append(1)
        else:
            y_pred.append(-1)

acc = accuracy_score(y_test, y_pred)
print(acc)


0.675
